# <div dir="auto">دورة تكوينية رقم 1: التنقيب في النصوص بلغة python  القرآن نموذجا </div>

## <div dir="auto"> المرحلة الأولى: الحصول على ملف أجزاء القرآن</div>

<div dir="auto">
في مقالة  <a  href="http://abdulbaqi.io/2019/01/15/quranic_roots/" >التالية</a>، يمكن استخدام  بعض أوامر `Linux` للبحث عن الكلمات الجذرية في القرآن الكريم بأكمله أو في سورة معينة من القرآن. على الرغم من أن أوامر `Linux` يمكن أن تكون فعّالة في بعض الحالات، إلا أنها ليست مصممة لتحليل البيانات. لذا، يتعين علينا اللجوء إلى لغة برمجة للعلوم البيانية مثل R أو Python.في هذه المقالة، سأستكشف قوة `Python` لمعالجة نفس الموضوع (أي الكلمات الجذرية في القرآن)، وسأوسع في حل المشكلة لتشمل استفسارات أكثر إثارة.
</div>

<div dir="auto">
لا تهدف هذه المقالة إلى أن تكون دورة تعليمية للمبتدئين لـ `python` أو `pandas`، الحزمة الخاصة بـ `python` المخصصة لتحليل البيانات والتي سأستخدمها هنا. أتوقع أن يكون لديك بعض الخبرة مع كل منهما، ولكن يمكن أن يكون لديك خبرة أقل مع `pandas` بشكل عام.
</div>

<div dir="auto">
بدون مزيد من التأخير، لنبدأ على بركة الله.
</div>

<div dir="auto">
أولاً، دعونا نبدأ ببعض الخطوات الأساسية مثل تحميل حزمة `pandas` وإعادة تسميتها لسهولة الاستخدام باسم `pd`.
</div>

In [1]:
#! pip install pandas 

In [2]:
import pandas as pd

<div dir="auto">
ثم، لقراءة الملف الذي يحتوي على المعلومات الصرفية. تحتوي مكتبة Pandas على وظيفة read_csv التي يمكنها القراءة مباشرة من عنوان URL. يرجى ملاحظة أن الملف يحتوي على بعض المعلومات حول حقوق الملكية في السطور الأولى 56، ولذا سنستخدم خيار skiprows. أيضًا، يجب مراعاة أن read_csv يفترض افتراضيًا أن المفصل بين الحقول هو الفاصلة، وإذا كان غير ذلك - كما هو الحال في هذا الملف - يجب علينا تحديد الفاصل بشكل صريح، وبالتالي خيار sep='\t'. وأخيرًا، نقوم بعرض بضعة أسطر من الجزء العلوي باستخدام وظيفة ()head.
</div>

In [3]:
url = 'http://textminingthequran.com/data/quranic-corpus-morphology-0.4.txt'
qdforiginal = pd.read_csv(url, sep='\t',skiprows=56)
qdforiginal.head()

,LOCATION,FORM,TAG,FEATURES
0,(1:1:1:1),bi,P,PREFIX|bi+
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN
3,(1:1:3:1),{l,DET,PREFIX|Al+
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN


<div dir="auto">
من الأحسن حفظ النسخة الأولى محليًا باستخدام وظيفة `()to_csv`.
</div>

In [4]:
qdforiginal.to_csv('quran-morphology-v1.csv')

<div dir="auto">
عند النظر إلى أول عدة أسطر في الملف أعلاه، نرى أن أعمدة `LOCATION` و `FEATURES` تحتاج إلى تقسيم إضافي.
</div>

<div dir="auto">
يحتوي ملفنا على أزيد من 128,000 سطر يمكنك التحقق من ذلك باستخدام الأمر `qdforiginal.shape`. أفضل أن نأخذ عينة صغيرة من هذا الملف الكبير ونقوم بتشغيل التجارب على عملية التقسيم. عند النجاح، يمكننا بعد ذلك تشغيلها على الملف بأكمله.
</div>

In [5]:
qdforiginal.shape

(128219, 4)

## <div dir="auto">  تقسيم الأعمدة</div>

<div dir="auto">
ٍنظرًا لأننا مهتمون  بالكلمات الجذرية، نريد تحديد كل الصفوف التي تحتوي على كلمة "ROOT:" في عمود "FEATURES". يمكن القيام بذلك باستخدام أمر مثل الأمر التالي:
</div>

In [6]:
qdforiginal.FEATURES.str.contains('ROOT')[:3]

0    False
1     True
2     True
Name: FEATURES, dtype: bool

<div dir="auto">
أخذنا الثلاثة أسطر الأولى فقط من إجمالي 128,219  سطر. يعيد ذلك قيم منطقية من نوع `True` أو `False`. لذلك، يمكننا تمرير نتيجة القيم المنطقية هذه لتصفية الإطار البيانات بأكمله بواسطة:
</div>

In [7]:
qdforiginal[qdforiginal.FEATURES.str.contains('ROOT:')].head(3)

,LOCATION,FORM,TAG,FEATURES
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN


<div dir="auto">
للحصول على عينة عشوائية، يمكننا دائمًا استخدام طريقة `()sample` على النحو التالي. سأسمي هذه العينة `qsample`.
</div>

In [ ]:
qsample = qdforiginal[qdforiginal.FEATURES.str.contains('ROOT:')].sample(10); qsample

<div dir="auto">
الهدف هو تقسيم عمود `LOCATION` إلى أربعة أعمدة، ثم تقسيم عمود `FEATURES` إلى عدة أعمدة كعمود للجذر .
</div>

<div dir="auto">
أولاً، سنقوم بتقسيم العمود الأول `LOCATION` إلى أربعة أعمدة. يتم ذلك من خلال استخدام وظيفة `extract` للسلسلة التي يمكنها أخذ تعبير عادي. يقوم بناء `<...>P?` ضمن التعبير العادي بإنشاء أعمدة بأسماء محددة.
</div>

In [8]:
tmp1 = qdforiginal.LOCATION.str.extract(r'(?P<sura>\d*):(?P<aya>\d*):(?P<word>\d*):(?P<w_seg>\d*)'); tmp1

,sura,aya,word,w_seg
0,1,1,1,1
1,1,1,1,2
2,1,1,2,1
3,1,1,3,1
4,1,1,3,2
...,...,...,...,...
128214,114,6,2,1
128215,114,6,2,2
128216,114,6,3,1
128217,114,6,3,2


<div dir="auto">
الآن، دعونا نستخرج الجذور من عمود `FEATURES` بنفس الطريقة.
</div>

In [9]:
tmp2 = qdforiginal.FEATURES.str.extract(r'ROOT:(?P<Root>[^|]*)')
tmp2.fillna('', inplace=True);tmp2

,Root
0,
1,smw
2,Alh
3,
4,rHm
...,...
128214,
128215,jnn
128216,
128217,


<div dir="auto">
للحصول على الكلمات بالحروف العربية سنستخدم حزمة تم تصميمها للتنقيب عن النصوص بسهولة وتسمى <a  href="https://github.com/annotation/text-fabric" >text-fabric</a>
</div>

In [10]:
#https://github.com/q-ran/quran/blob/master/programs/morph.ipynb
from tf.writing.transcription import Transcription as tr
tmp2['RootAr'] = tmp2['Root'].map(lambda x:  tr.to_arabic(str(x)) if x!= ''  else x);tmp2

,Root,RootAr
0,,
1,smw,سمو
2,Alh,اله
3,,
4,rHm,رحم
...,...,...
128214,,
128215,jnn,جنن
128216,,
128217,,


<div dir="auto">
بالمثل، سنستخراج الأشكال الأصلية (Lemmas) أيضًا.
</div>

In [11]:
tmp3 = qdforiginal.FEATURES.str.extract(r'LEM:(?P<Lemma>[^|]*)'); 
tmp3.fillna('', inplace=True)
tmp3['LemmaAr'] = tmp3['Lemma'].map(lambda x:  tr.to_arabic(str(x)));tmp3

,Lemma,LemmaAr
0,,
1,{som,ٱسْم
2,{ll~ah,ٱللَّه
3,,
4,r~aHoma`n,رَّحْمَٰن
...,...,...
128214,,
128215,jin~ap,جِنَّة
128216,,
128217,,


<div dir="auto">
كما يمكننا اضافة العمود الجديد مباشرة على الشكل التالي:
</div>

In [12]:
qdforiginal['FORMAr'] = qdforiginal['FORM'].map(lambda x: tr.to_arabic(str(x)))

In [13]:
qdforiginal

,LOCATION,FORM,TAG,FEATURES,FORMAr
0,(1:1:1:1),bi,P,PREFIX|bi+,بِ
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,سْمِ
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN,ٱللَّهِ
3,(1:1:3:1),{l,DET,PREFIX|Al+,ٱل
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,رَّحْمَٰنِ
...,...,...,...,...,...
128214,(114:6:2:1),{lo,DET,PREFIX|Al+,ٱلْ
128215,(114:6:2:2),jin~api,N,STEM|POS:N|LEM:jin~ap|ROOT:jnn|F|GEN,جِنَّةِ
128216,(114:6:3:1),wa,CONJ,PREFIX|w:CONJ+,وَ
128217,(114:6:3:2),{l,DET,PREFIX|Al+,ٱل


<div dir="auto">
أخيرًا في هذه المرحلة الأولى، كل ما تبقى هو دمج `qdforiginal` مع هذه الثلاثة تقسيمات `tmp1، tmp2، tmp3`، على النحو التالي. الخيار `axis=1` يعني تشغيل الدمج على الأعمدة (وليس الصفوف).
</div>

In [14]:
df_qruan=pd.concat([tmp1, qdforiginal, tmp2, tmp3], axis=1); df_qruan

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,1,1,1,1,(1:1:1:1),bi,P,PREFIX|bi+,بِ,,,,
1,1,1,1,2,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,سْمِ,smw,سمو,{som,ٱسْم
2,1,1,2,1,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN,ٱللَّهِ,Alh,اله,{ll~ah,ٱللَّه
3,1,1,3,1,(1:1:3:1),{l,DET,PREFIX|Al+,ٱل,,,,
4,1,1,3,2,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,رَّحْمَٰنِ,rHm,رحم,r~aHoma`n,رَّحْمَٰن
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128214,114,6,2,1,(114:6:2:1),{lo,DET,PREFIX|Al+,ٱلْ,,,,
128215,114,6,2,2,(114:6:2:2),jin~api,N,STEM|POS:N|LEM:jin~ap|ROOT:jnn|F|GEN,جِنَّةِ,jnn,جنن,jin~ap,جِنَّة
128216,114,6,3,1,(114:6:3:1),wa,CONJ,PREFIX|w:CONJ+,وَ,,,,
128217,114,6,3,2,(114:6:3:2),{l,DET,PREFIX|Al+,ٱل,,,,


<div dir="auto">
لحفظ النسخة يجب إضافة التشقير التالي 'utf-8-sig'. 
</div>

In [15]:
df_qruan.to_csv('quran-morphology-v2.csv', encoding='utf-8-sig')

## <div dir="auto">  المرحلة الثانية: انشاء قاعدة البيانات للقرآن</div>

In [16]:
import sqlite3 as sql

In [17]:
### --- CREATING THE DATABASE + CURSOR
conn = sql.connect('QAC_db.db')
cur = conn.cursor()

In [ ]:
#Doping QAC table if already exists
#cur.execute("DROP TABLE QAC_TABLE")

In [18]:
### --- CREATING THE TABLE
cur.execute('''CREATE TABLE IF NOT EXISTS QAC_TABLE
           (unique_id TEXT PRIMARY KEY, sura INTEGER, aya INTEGER, word INTEGER, w_seg INTEGER, LOCATION TEXT, FORM TEXT, TAG TEXT, FEATURES TEXT, ROOT TEXT, Lemma TEXT, FORMAr TEXT,ROOTAr TEXT, LemmaAr TEXT)''')


In [19]:
### --- COPYING THE DATAFRAME TO THE TABLE
df_qruan.to_sql('QAC_TABLE', conn, if_exists='replace', index=False)

128219

In [20]:
pd.read_sql('select * from QAC_TABLE', conn)

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,1,1,1,1,(1:1:1:1),bi,P,PREFIX|bi+,بِ,,,,
1,1,1,1,2,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,سْمِ,smw,سمو,{som,ٱسْم
2,1,1,2,1,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN,ٱللَّهِ,Alh,اله,{ll~ah,ٱللَّه
3,1,1,3,1,(1:1:3:1),{l,DET,PREFIX|Al+,ٱل,,,,
4,1,1,3,2,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,رَّحْمَٰنِ,rHm,رحم,r~aHoma`n,رَّحْمَٰن
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128214,114,6,2,1,(114:6:2:1),{lo,DET,PREFIX|Al+,ٱلْ,,,,
128215,114,6,2,2,(114:6:2:2),jin~api,N,STEM|POS:N|LEM:jin~ap|ROOT:jnn|F|GEN,جِنَّةِ,jnn,جنن,jin~ap,جِنَّة
128216,114,6,3,1,(114:6:3:1),wa,CONJ,PREFIX|w:CONJ+,وَ,,,,
128217,114,6,3,2,(114:6:3:2),{l,DET,PREFIX|Al+,ٱل,,,,


# Verb Features
## Aspect features
PERF 	فعل ماض 	Perfect verb<br>
IMPF 	فعل مضارع 	Imperfect verb<br>
IMPV 	فعل أمر 	Imperative verb<br>

In [21]:
seev = pd.read_sql("select * from QAC_TABLE where TAG= 'V' and FEATURES LIKE '%IMPF%' and FEATURES LIKE '%PASS%' and FEATURES LIKE '%MOOD%' and FEATURES LIKE '%(II)%'", conn)
with pd.option_context('display.max_colwidth', None):
  display(seev.head())


,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,2,96,21,1,(2:96:21:1),yuEam~ara,V,STEM|POS:V|IMPF|PASS|(II)|LEM:yuEam~aru|ROOT:Emr|3MS|MOOD:SUBJ,يُعَمَّرَ,Emr,عمر,yuEam~aru,يُعَمَّرُ
1,2,105,11,1,(2:105:11:1),yunaz~ala,V,STEM|POS:V|IMPF|PASS|(II)|LEM:naz~ala|ROOT:nzl|3MS|MOOD:SUBJ,يُنَزَّلَ,nzl,نزل,naz~ala,نَزَّلَ
2,2,272,24,1,(2:272:24:1),yuwaf~a,V,STEM|POS:V|IMPF|PASS|(II)|LEM:waf~aY`^|ROOT:wfy|3MS|MOOD:JUS,يُوَفَّ,wfy,وفي,waf~aY`^,وَفَّىٰٓ
3,3,93,16,1,(3:93:16:1),tunaz~ala,V,STEM|POS:V|IMPF|PASS|(II)|LEM:naz~ala|ROOT:nzl|3FS|MOOD:SUBJ,تُنَزَّلَ,nzl,نزل,naz~ala,نَزَّلَ
4,5,33,12,1,(5:33:12:1),yuqat~alu,V,STEM|POS:V|IMPF|PASS|(II)|LEM:qut~ilu|ROOT:qtl|3MP|MOOD:SUBJ,يُقَتَّلُ,qtl,قتل,qut~ilu,قُتِّلُ


In [22]:
allv=pd.read_sql("select * from QAC_TABLE where TAG ='V'", conn)
with pd.option_context('display.max_colwidth', None):
  display(allv.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ


In [23]:
allv['FEATURES2']=[x.split('STEM|POS:V|')[1] for x in allv['FEATURES']]
with pd.option_context('display.max_colwidth', None):
  display(allv.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF|LEM:Eabada|ROOT:Ebd|1P
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV|LEM:hadaY|ROOT:hdy|2MS
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP


In [24]:
allv['Aspect']=[x.split('|')[0] for x in allv['FEATURES2']]
with pd.option_context('display.max_colwidth', None):
  display(allv.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Aspect
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF|LEM:Eabada|ROOT:Ebd|1P,IMPF
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,IMPF
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV|LEM:hadaY|ROOT:hdy|2MS,IMPV
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,PERF
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,IMPF


##  Voice features
ACT 	مبني للمعلوم 	Active voice (default)<br>
PASS 	مبني للمجهول 	Passive voice<br>

In [25]:
allv['Voice']=[x.split('|')[1] if 'PASS' in x else 'ACT' for x in allv['FEATURES2']]; allv

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Aspect,Voice
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF|LEM:Eabada|ROOT:Ebd|1P,IMPF,ACT
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,IMPF,ACT
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV|LEM:hadaY|ROOT:hdy|2MS,IMPV,ACT
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,PERF,ACT
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,IMPF,ACT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,113,3,5,1,(113:3:5:1),waqaba,V,STEM|POS:V|PERF|LEM:waqaba|ROOT:wqb|3MS,وَقَبَ,wqb,وقب,waqaba,وَقَبَ,PERF|LEM:waqaba|ROOT:wqb|3MS,PERF,ACT
19352,113,5,5,1,(113:5:5:1),Hasada,V,STEM|POS:V|PERF|LEM:Hasada|ROOT:Hsd|3MS,حَسَدَ,Hsd,حسد,Hasada,حَسَدَ,PERF|LEM:Hasada|ROOT:Hsd|3MS,PERF,ACT
19353,114,1,1,1,(114:1:1:1),qulo,V,STEM|POS:V|IMPV|LEM:qaAla|ROOT:qwl|2MS,قُلْ,qwl,قول,qaAla,قَالَ,IMPV|LEM:qaAla|ROOT:qwl|2MS,IMPV,ACT
19354,114,1,2,1,(114:1:2:1),>aEuw*u,V,STEM|POS:V|IMPF|LEM:Eu*o|ROOT:Ew*|1S,أَعُوذُ,Ew*,عوذ,Eu*o,عُذْ,IMPF|LEM:Eu*o|ROOT:Ew*|1S,IMPF,ACT


## Mood features
IND 	مرفوع 	Indicative mood (default)<br>
SUBJ 	منصوب 	Subjunctive mood<br>
JUS 	مجزوم 	Jussive mood<br>

In [26]:
allv['MOOD']=allv['FEATURES'].map(lambda x:  x.split('|MOOD:')[1] if '|MOOD:' in x  else 'IND'); allv

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Aspect,Voice,MOOD
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF|LEM:Eabada|ROOT:Ebd|1P,IMPF,ACT,IND
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,IMPF,ACT,IND
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV|LEM:hadaY|ROOT:hdy|2MS,IMPV,ACT,IND
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,PERF,ACT,IND
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,IMPF,ACT,IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,113,3,5,1,(113:3:5:1),waqaba,V,STEM|POS:V|PERF|LEM:waqaba|ROOT:wqb|3MS,وَقَبَ,wqb,وقب,waqaba,وَقَبَ,PERF|LEM:waqaba|ROOT:wqb|3MS,PERF,ACT,IND
19352,113,5,5,1,(113:5:5:1),Hasada,V,STEM|POS:V|PERF|LEM:Hasada|ROOT:Hsd|3MS,حَسَدَ,Hsd,حسد,Hasada,حَسَدَ,PERF|LEM:Hasada|ROOT:Hsd|3MS,PERF,ACT,IND
19353,114,1,1,1,(114:1:1:1),qulo,V,STEM|POS:V|IMPV|LEM:qaAla|ROOT:qwl|2MS,قُلْ,qwl,قول,qaAla,قَالَ,IMPV|LEM:qaAla|ROOT:qwl|2MS,IMPV,ACT,IND
19354,114,1,2,1,(114:1:2:1),>aEuw*u,V,STEM|POS:V|IMPF|LEM:Eu*o|ROOT:Ew*|1S,أَعُوذُ,Ew*,عوذ,Eu*o,عُذْ,IMPF|LEM:Eu*o|ROOT:Ew*|1S,IMPF,ACT,IND


https://corpus.quran.com/documentation/verbforms.jsp
# Verb form features
I 	فَعَلَ First form (default)<br>
II 	فَعَّلَ Second form<br>
III 	فَاعَلَ Third form<br>
IV 	أَفْعَلَ Fourth form<br>
V 	تَفَعَّلَ Fifth form<br>
VI 	تَفَاعَلَ Sixth form<br>
VII 	إِنْفَعَلَ Seventh form<br>
VIII 	إِفْتَعَلَ Eighth form<br>
IX 	إِفْعَلَّ Ninth form<br>
X 	إِسْتَفْعَلَ Tenth form<br>
XI 	Eleventh form<br>
XII 	Twelfth form

In [27]:
import re
allv['FormV']=[re.findall(re.escape('|(')+"(.*)"+re.escape(')'),x)[0] if '|(' in x else 'I' for x in allv['FEATURES']]; allv

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Aspect,Voice,MOOD,FormV
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF|LEM:Eabada|ROOT:Ebd|1P,IMPF,ACT,IND,I
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,IMPF,ACT,IND,X
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV|LEM:hadaY|ROOT:hdy|2MS,IMPV,ACT,IND,I
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,PERF,ACT,IND,IV
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,IMPF,ACT,IND,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,113,3,5,1,(113:3:5:1),waqaba,V,STEM|POS:V|PERF|LEM:waqaba|ROOT:wqb|3MS,وَقَبَ,wqb,وقب,waqaba,وَقَبَ,PERF|LEM:waqaba|ROOT:wqb|3MS,PERF,ACT,IND,I
19352,113,5,5,1,(113:5:5:1),Hasada,V,STEM|POS:V|PERF|LEM:Hasada|ROOT:Hsd|3MS,حَسَدَ,Hsd,حسد,Hasada,حَسَدَ,PERF|LEM:Hasada|ROOT:Hsd|3MS,PERF,ACT,IND,I
19353,114,1,1,1,(114:1:1:1),qulo,V,STEM|POS:V|IMPV|LEM:qaAla|ROOT:qwl|2MS,قُلْ,qwl,قول,qaAla,قَالَ,IMPV|LEM:qaAla|ROOT:qwl|2MS,IMPV,ACT,IND,I
19354,114,1,2,1,(114:1:2:1),>aEuw*u,V,STEM|POS:V|IMPF|LEM:Eu*o|ROOT:Ew*|1S,أَعُوذُ,Ew*,عوذ,Eu*o,عُذْ,IMPF|LEM:Eu*o|ROOT:Ew*|1S,IMPF,ACT,IND,I


## Person, Gender and Number

person 	الاسناد 	1, 2, 3 	first person, second person, third person<br>
gender 	الجنس 	M, F 	masculine, feminine<br>
number 	العدد 	S, D, P 	singular, dual, plural

In [28]:
allv['FEATURESR']=allv['FEATURES'].map(lambda x:  x.split('|MOOD:')[0] if '|MOOD:' in x  else x)
allv['PGN']=[x.split('|')[-1] for x in allv['FEATURESR']];allv


,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Aspect,Voice,MOOD,FormV,FEATURESR,PGN
0,1,5,2,1,(1:5:2:1),naEobudu,V,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF|LEM:Eabada|ROOT:Ebd|1P,IMPF,ACT,IND,I,STEM|POS:V|IMPF|LEM:Eabada|ROOT:Ebd|1P,1P
1,1,5,4,1,(1:5:4:1),nasotaEiynu,V,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,IMPF,ACT,IND,X,STEM|POS:V|IMPF|(X)|LEM:{sotaEiynu|ROOT:Ewn|1P,1P
2,1,6,1,1,(1:6:1:1),{hodi,V,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV|LEM:hadaY|ROOT:hdy|2MS,IMPV,ACT,IND,I,STEM|POS:V|IMPV|LEM:hadaY|ROOT:hdy|2MS,2MS
3,1,7,3,1,(1:7:3:1),>anoEamo,V,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,PERF,ACT,IND,IV,STEM|POS:V|PERF|(IV)|LEM:>anoEama|ROOT:nEm|2MS,2MS
4,2,3,2,1,(2:3:2:1),yu&ominu,V,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,IMPF,ACT,IND,IV,STEM|POS:V|IMPF|(IV)|LEM:'aAmana|ROOT:Amn|3MP,3MP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,113,3,5,1,(113:3:5:1),waqaba,V,STEM|POS:V|PERF|LEM:waqaba|ROOT:wqb|3MS,وَقَبَ,wqb,وقب,waqaba,وَقَبَ,PERF|LEM:waqaba|ROOT:wqb|3MS,PERF,ACT,IND,I,STEM|POS:V|PERF|LEM:waqaba|ROOT:wqb|3MS,3MS
19352,113,5,5,1,(113:5:5:1),Hasada,V,STEM|POS:V|PERF|LEM:Hasada|ROOT:Hsd|3MS,حَسَدَ,Hsd,حسد,Hasada,حَسَدَ,PERF|LEM:Hasada|ROOT:Hsd|3MS,PERF,ACT,IND,I,STEM|POS:V|PERF|LEM:Hasada|ROOT:Hsd|3MS,3MS
19353,114,1,1,1,(114:1:1:1),qulo,V,STEM|POS:V|IMPV|LEM:qaAla|ROOT:qwl|2MS,قُلْ,qwl,قول,qaAla,قَالَ,IMPV|LEM:qaAla|ROOT:qwl|2MS,IMPV,ACT,IND,I,STEM|POS:V|IMPV|LEM:qaAla|ROOT:qwl|2MS,2MS
19354,114,1,2,1,(114:1:2:1),>aEuw*u,V,STEM|POS:V|IMPF|LEM:Eu*o|ROOT:Ew*|1S,أَعُوذُ,Ew*,عوذ,Eu*o,عُذْ,IMPF|LEM:Eu*o|ROOT:Ew*|1S,IMPF,ACT,IND,I,STEM|POS:V|IMPF|LEM:Eu*o|ROOT:Ew*|1S,1S


In [29]:
allv.drop(['LOCATION', 'FEATURES', 'FEATURES2', 'FEATURESR'], axis=1, inplace=True); allv

,sura,aya,word,w_seg,FORM,TAG,FORMAr,Root,RootAr,Lemma,LemmaAr,Aspect,Voice,MOOD,FormV,PGN
0,1,5,2,1,naEobudu,V,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF,ACT,IND,I,1P
1,1,5,4,1,nasotaEiynu,V,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF,ACT,IND,X,1P
2,1,6,1,1,{hodi,V,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV,ACT,IND,I,2MS
3,1,7,3,1,>anoEamo,V,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF,ACT,IND,IV,2MS
4,2,3,2,1,yu&ominu,V,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF,ACT,IND,IV,3MP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,113,3,5,1,waqaba,V,وَقَبَ,wqb,وقب,waqaba,وَقَبَ,PERF,ACT,IND,I,3MS
19352,113,5,5,1,Hasada,V,حَسَدَ,Hsd,حسد,Hasada,حَسَدَ,PERF,ACT,IND,I,3MS
19353,114,1,1,1,qulo,V,قُلْ,qwl,قول,qaAla,قَالَ,IMPV,ACT,IND,I,2MS
19354,114,1,2,1,>aEuw*u,V,أَعُوذُ,Ew*,عوذ,Eu*o,عُذْ,IMPF,ACT,IND,I,1S


In [32]:
### --- CREATING THE TABLE
cur.execute('''CREATE TABLE IF NOT EXISTS QACV_TABLE
           (unique_id TEXT PRIMARY KEY, sura INTEGER, aya INTEGER, word INTEGER, w_seg INTEGER,  FORM TEXT, TAG TEXT, ROOT TEXT, Lemma TEXT, FORMAr TEXT,ROOTAr TEXT, LemmaAr TEXT, Aspect 	TEXT, MOOD TEXT, Voice TEXT, FormV TEXT, PGN TEXT)''')


In [33]:
### --- COPYING THE DATAFRAME TO THE TABLE
allv.to_sql('QACV_TABLE', conn, if_exists='replace', index=False)

19356

In [34]:
pd.read_sql('select * from QACV_TABLE', conn)

,sura,aya,word,w_seg,FORM,TAG,FORMAr,Root,RootAr,Lemma,LemmaAr,Aspect,Voice,MOOD,FormV,PGN
0,1,5,2,1,naEobudu,V,نَعْبُدُ,Ebd,عبد,Eabada,عَبَدَ,IMPF,ACT,IND,I,1P
1,1,5,4,1,nasotaEiynu,V,نَسْتَعِينُ,Ewn,عون,{sotaEiynu,ٱسْتَعِينُ,IMPF,ACT,IND,X,1P
2,1,6,1,1,{hodi,V,ٱهْدِ,hdy,هدي,hadaY,هَدَى,IMPV,ACT,IND,I,2MS
3,1,7,3,1,>anoEamo,V,أَنْعَمْ,nEm,نعم,>anoEama,أَنْعَمَ,PERF,ACT,IND,IV,2MS
4,2,3,2,1,yu&ominu,V,يُؤْمِنُ,Amn,امن,'aAmana,ءَامَنَ,IMPF,ACT,IND,IV,3MP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,113,3,5,1,waqaba,V,وَقَبَ,wqb,وقب,waqaba,وَقَبَ,PERF,ACT,IND,I,3MS
19352,113,5,5,1,Hasada,V,حَسَدَ,Hsd,حسد,Hasada,حَسَدَ,PERF,ACT,IND,I,3MS
19353,114,1,1,1,qulo,V,قُلْ,qwl,قول,qaAla,قَالَ,IMPV,ACT,IND,I,2MS
19354,114,1,2,1,>aEuw*u,V,أَعُوذُ,Ew*,عوذ,Eu*o,عُذْ,IMPF,ACT,IND,I,1S


# Derived Nouns:
Noun (N) or adjective (ADJ) صفة part-of speech-tag with:

In [35]:
seenad=pd.read_sql("select * from QAC_TABLE where  FEATURES Like '%N|PASS%' or FEATURES Like '%ADJ%'", conn)
with pd.option_context('display.max_colwidth', None):
  display(seenad.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,1,1,3,2,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,رَّحْمَٰنِ,rHm,رحم,r~aHoma`n,رَّحْمَٰن
1,1,1,4,2,(1:1:4:2),r~aHiymi,ADJ,STEM|POS:ADJ|LEM:r~aHiym|ROOT:rHm|MS|GEN,رَّحِيمِ,rHm,رحم,r~aHiym,رَّحِيم
2,1,3,1,2,(1:3:1:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN,رَّحْمَٰنِ,rHm,رحم,r~aHoma`n,رَّحْمَٰن
3,1,3,2,2,(1:3:2:2),r~aHiymi,ADJ,STEM|POS:ADJ|LEM:r~aHiym|ROOT:rHm|MS|GEN,رَّحِيمِ,rHm,رحم,r~aHiym,رَّحِيم
4,1,6,3,2,(1:6:3:2),musotaqiyma,ADJ,STEM|POS:ADJ|ACT|PCPL|(X)|LEM:m~usotaqiym|ROOT:qwm|M|ACC,مُسْتَقِيمَ,qwm,قوم,m~usotaqiym,مُّسْتَقِيم


## Derivation features
ACT PCPL 	اسم فاعل 	Active participle <br>
PASS PCPL 	اسم مفعول 	Passive participle<br>
VN 	مصدر 	Verbal noun

In [36]:
alln=pd.read_sql("select * from QAC_TABLE where TAG='N' and (FEATURES Like '%PCPL%' or FEATURES Like '%VN%') ", conn)
with pd.option_context('display.max_colwidth', None):
  display(alln.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:wqy|MP|GEN,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:flH|MP|NOM,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون


In [37]:
alladj=pd.read_sql("select * from QAC_TABLE where TAG='ADJ' and (FEATURES Like '%PCPL%' or FEATURES Like '%VN%') ", conn)

In [38]:
import re
alln['FEATURES2']=[re.findall(re.escape('STEM|POS:')+"(.*)"+re.escape('|LEM:'),x)[0] for x in alln['FEATURES']]
alln

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,N|ACT|PCPL
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,N|PASS|PCPL
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,N|ACT|PCPL
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:...,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,N|ACT|PCPL|(VIII)
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:f...,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,N|ACT|PCPL|(IV)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,(109:5:3:1),Ea`biduwna,N,STEM|POS:N|ACT|PCPL|LEM:EaAbid|ROOT:Ebd|MP|NOM,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,N|ACT|PCPL
3865,110,3,7,1,(110:3:7:1),taw~aAbF[A,N,STEM|POS:N|ACT|PCPL|LEM:taw~aAb|ROOT:twb|MS|IN...,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,N|ACT|PCPL
3866,111,4,2,1,(111:4:2:1),Ham~aAlapa,N,STEM|POS:N|ACT|PCPL|LEM:Ham~aAlap|ROOT:Hml|FS|ACC,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,N|ACT|PCPL
3867,113,3,3,1,(113:3:3:1),gaAsiqK,N,STEM|POS:N|ACT|PCPL|LEM:gaAsiq|ROOT:gsq|M|INDE...,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,N|ACT|PCPL


In [39]:
alladj['FEATURES2']=[re.findall(re.escape('STEM|POS:')+"(.*)"+re.escape('|LEM:'),x)[0] for x in alladj['FEATURES']]

In [40]:
alln['Derivation']=alln['FEATURES2'].map(lambda x:  x.split('|')[1]+' '+'PCPL' if '|PCPL' in x  else 'VN'); alln

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Derivation
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,N|ACT|PCPL,ACT PCPL
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,N|PASS|PCPL,PASS PCPL
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,N|ACT|PCPL,ACT PCPL
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:...,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,N|ACT|PCPL|(VIII),ACT PCPL
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:f...,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,N|ACT|PCPL|(IV),ACT PCPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,(109:5:3:1),Ea`biduwna,N,STEM|POS:N|ACT|PCPL|LEM:EaAbid|ROOT:Ebd|MP|NOM,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,N|ACT|PCPL,ACT PCPL
3865,110,3,7,1,(110:3:7:1),taw~aAbF[A,N,STEM|POS:N|ACT|PCPL|LEM:taw~aAb|ROOT:twb|MS|IN...,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,N|ACT|PCPL,ACT PCPL
3866,111,4,2,1,(111:4:2:1),Ham~aAlapa,N,STEM|POS:N|ACT|PCPL|LEM:Ham~aAlap|ROOT:Hml|FS|ACC,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,N|ACT|PCPL,ACT PCPL
3867,113,3,3,1,(113:3:3:1),gaAsiqK,N,STEM|POS:N|ACT|PCPL|LEM:gaAsiq|ROOT:gsq|M|INDE...,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,N|ACT|PCPL,ACT PCPL


In [41]:
alladj['Derivation']=alladj['FEATURES2'].map(lambda x:  x.split('|')[1]+' '+'PCPL' if '|PCPL' in x  else 'VN')

# Nominal Features:
## State features
DEF 	معرفة 	Definite state; <br>
INDEF 	نكرة 	Indefinite state

In [42]:
indef=pd.read_sql("select * from QAC_TABLE where TAG ='N' and FEATURES NOT Like '%INDEF%' ", conn)
with pd.option_context('display.max_colwidth', None):
  display(indef.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr
0,1,1,1,2,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN,سْمِ,smw,سمو,{som,ٱسْم
1,1,2,1,2,(1:2:1:2),Hamodu,N,STEM|POS:N|LEM:Hamod|ROOT:Hmd|M|NOM,حَمْدُ,Hmd,حمد,Hamod,حَمْد
2,1,2,3,1,(1:2:3:1),rab~i,N,STEM|POS:N|LEM:rab~|ROOT:rbb|M|GEN,رَبِّ,rbb,ربب,rab~,رَبّ
3,1,2,4,2,(1:2:4:2),Ea`lamiyna,N,STEM|POS:N|LEM:Ea`lamiyn|ROOT:Elm|MP|GEN,عَٰلَمِينَ,Elm,علم,Ea`lamiyn,عَٰلَمِين
4,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك


In [43]:
alln['State']=alln['FEATURES'].map(lambda x:  'INDEF' if '|INDEF' in x  else 'DEF'); alln

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Derivation,State
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,N|ACT|PCPL,ACT PCPL,DEF
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,N|PASS|PCPL,PASS PCPL,DEF
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,N|ACT|PCPL,ACT PCPL,DEF
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:...,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,N|ACT|PCPL|(VIII),ACT PCPL,DEF
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:f...,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,N|ACT|PCPL|(IV),ACT PCPL,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,(109:5:3:1),Ea`biduwna,N,STEM|POS:N|ACT|PCPL|LEM:EaAbid|ROOT:Ebd|MP|NOM,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,N|ACT|PCPL,ACT PCPL,DEF
3865,110,3,7,1,(110:3:7:1),taw~aAbF[A,N,STEM|POS:N|ACT|PCPL|LEM:taw~aAb|ROOT:twb|MS|IN...,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,N|ACT|PCPL,ACT PCPL,INDEF
3866,111,4,2,1,(111:4:2:1),Ham~aAlapa,N,STEM|POS:N|ACT|PCPL|LEM:Ham~aAlap|ROOT:Hml|FS|ACC,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,N|ACT|PCPL,ACT PCPL,DEF
3867,113,3,3,1,(113:3:3:1),gaAsiqK,N,STEM|POS:N|ACT|PCPL|LEM:gaAsiq|ROOT:gsq|M|INDE...,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,N|ACT|PCPL,ACT PCPL,INDEF


In [44]:
alladj['State']=alladj['FEATURES'].map(lambda x:  'INDEF' if '|INDEF' in x  else 'DEF')

## Case features
NOM 	مرفوع 	Nominative case;<br>
ACC 	منصوب 	Accusative case;<br>
GEN 	مجرور 	Genitive case

extract string between two strings python

In [45]:
alln['FEATURESR']=[x.split('ROOT:')[1] for x in alln['FEATURES']]
alln['FEATURESR']=[x.split('|')[1:] for x in alln['FEATURESR']]
with pd.option_context('display.max_colwidth', None):
  display(alln.head())

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Derivation,State,FEATURESR
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,N|ACT|PCPL,ACT PCPL,DEF,"[M, GEN]"
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,N|PASS|PCPL,PASS PCPL,DEF,"[M, GEN]"
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,N|ACT|PCPL,ACT PCPL,DEF,"[MP, GEN]"
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:wqy|MP|GEN,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,N|ACT|PCPL|(VIII),ACT PCPL,DEF,"[MP, GEN]"
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:flH|MP|NOM,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,N|ACT|PCPL|(IV),ACT PCPL,DEF,"[MP, NOM]"


In [46]:
alladj['FEATURESR']=[x.split('ROOT:')[1] for x in alladj['FEATURES']]
alladj['FEATURESR']=[x.split('|')[1:] for x in alladj['FEATURESR']]

In [47]:
crx=alladj[alladj['LOCATION'] == '(2:69:15:1)'] 
with pd.option_context('display.max_colwidth', None):
  display(crx)

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Derivation,State,FEATURESR
4,2,69,15,1,(2:69:15:1),faAqiEN,ADJ,STEM|POS:ADJ|ACT|PCPL|LEM:faAqiE|ROOT:fqE|INDEF|NOM,فَاقِعٌ,fqE,فقع,faAqiE,فَاقِع,ADJ|ACT|PCPL,ACT PCPL,INDEF,"[INDEF, NOM]"


In [48]:
alln['Grop']=[x[0] if 'INDEF' not in str(x) and len(x)>1 else '' for x in alln['FEATURESR']]
alln['Case']=[str(x[-1]) for x in alln['FEATURESR']]
alln

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Derivation,State,FEATURESR,Grop,Case
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,N|ACT|PCPL,ACT PCPL,DEF,"[M, GEN]",M,GEN
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,N|PASS|PCPL,PASS PCPL,DEF,"[M, GEN]",M,GEN
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,N|ACT|PCPL,ACT PCPL,DEF,"[MP, GEN]",MP,GEN
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:...,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,N|ACT|PCPL|(VIII),ACT PCPL,DEF,"[MP, GEN]",MP,GEN
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:f...,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,N|ACT|PCPL|(IV),ACT PCPL,DEF,"[MP, NOM]",MP,NOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,(109:5:3:1),Ea`biduwna,N,STEM|POS:N|ACT|PCPL|LEM:EaAbid|ROOT:Ebd|MP|NOM,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,N|ACT|PCPL,ACT PCPL,DEF,"[MP, NOM]",MP,NOM
3865,110,3,7,1,(110:3:7:1),taw~aAbF[A,N,STEM|POS:N|ACT|PCPL|LEM:taw~aAb|ROOT:twb|MS|IN...,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,N|ACT|PCPL,ACT PCPL,INDEF,"[MS, INDEF, ACC]",,ACC
3866,111,4,2,1,(111:4:2:1),Ham~aAlapa,N,STEM|POS:N|ACT|PCPL|LEM:Ham~aAlap|ROOT:Hml|FS|ACC,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,N|ACT|PCPL,ACT PCPL,DEF,"[FS, ACC]",FS,ACC
3867,113,3,3,1,(113:3:3:1),gaAsiqK,N,STEM|POS:N|ACT|PCPL|LEM:gaAsiq|ROOT:gsq|M|INDE...,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,N|ACT|PCPL,ACT PCPL,INDEF,"[M, INDEF, GEN]",,GEN


In [49]:
alladj['Grop']=[x[0] if 'INDEF' not in str(x) else '' for x in alladj['FEATURESR']]
alladj['Case']=[str(x[-1]) for x in alladj['FEATURESR']]

In [50]:
alln['FormN']=[re.findall(re.escape('|(')+"(.*)"+re.escape(')'),x)[0] if '|(' in x else 'I' for x in alln['FEATURES']]
alln

,sura,aya,word,w_seg,LOCATION,FORM,TAG,FEATURES,FORMAr,Root,RootAr,Lemma,LemmaAr,FEATURES2,Derivation,State,FEATURESR,Grop,Case,FormN
0,1,4,1,1,(1:4:1:1),ma`liki,N,STEM|POS:N|ACT|PCPL|LEM:ma`lik|ROOT:mlk|M|GEN,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,N|ACT|PCPL,ACT PCPL,DEF,"[M, GEN]",M,GEN,I
1,1,7,6,2,(1:7:6:2),magoDuwbi,N,STEM|POS:N|PASS|PCPL|LEM:magoDuwb|ROOT:gDb|M|GEN,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,N|PASS|PCPL,PASS PCPL,DEF,"[M, GEN]",M,GEN,I
2,1,7,9,2,(1:7:9:2),D~aA^l~iyna,N,STEM|POS:N|ACT|PCPL|LEM:DaA^l~|ROOT:Dll|MP|GEN,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,N|ACT|PCPL,ACT PCPL,DEF,"[MP, GEN]",MP,GEN,I
3,2,2,7,3,(2:2:7:3),mut~aqiyna,N,STEM|POS:N|ACT|PCPL|(VIII)|LEM:mut~aqiyn|ROOT:...,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,N|ACT|PCPL|(VIII),ACT PCPL,DEF,"[MP, GEN]",MP,GEN,VIII
4,2,5,8,2,(2:5:8:2),mufoliHuwna,N,STEM|POS:N|ACT|PCPL|(IV)|LEM:mufoliHuwn|ROOT:f...,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,N|ACT|PCPL|(IV),ACT PCPL,DEF,"[MP, NOM]",MP,NOM,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,(109:5:3:1),Ea`biduwna,N,STEM|POS:N|ACT|PCPL|LEM:EaAbid|ROOT:Ebd|MP|NOM,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,N|ACT|PCPL,ACT PCPL,DEF,"[MP, NOM]",MP,NOM,I
3865,110,3,7,1,(110:3:7:1),taw~aAbF[A,N,STEM|POS:N|ACT|PCPL|LEM:taw~aAb|ROOT:twb|MS|IN...,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,N|ACT|PCPL,ACT PCPL,INDEF,"[MS, INDEF, ACC]",,ACC,I
3866,111,4,2,1,(111:4:2:1),Ham~aAlapa,N,STEM|POS:N|ACT|PCPL|LEM:Ham~aAlap|ROOT:Hml|FS|ACC,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,N|ACT|PCPL,ACT PCPL,DEF,"[FS, ACC]",FS,ACC,I
3867,113,3,3,1,(113:3:3:1),gaAsiqK,N,STEM|POS:N|ACT|PCPL|LEM:gaAsiq|ROOT:gsq|M|INDE...,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,N|ACT|PCPL,ACT PCPL,INDEF,"[M, INDEF, GEN]",,GEN,I


In [51]:
alladj['FormN']=[re.findall(re.escape('|(')+"(.*)"+re.escape(')'),x)[0] if '|(' in x else 'I' for x in alladj['FEATURES']]

In [52]:
alln.drop(['LOCATION', 'FEATURES', 'FEATURES2', 'FEATURESR'], axis=1, inplace=True); alln

,sura,aya,word,w_seg,FORM,TAG,FORMAr,Root,RootAr,Lemma,LemmaAr,Derivation,State,Grop,Case,FormN
0,1,4,1,1,ma`liki,N,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,ACT PCPL,DEF,M,GEN,I
1,1,7,6,2,magoDuwbi,N,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,PASS PCPL,DEF,M,GEN,I
2,1,7,9,2,D~aA^l~iyna,N,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,ACT PCPL,DEF,MP,GEN,I
3,2,2,7,3,mut~aqiyna,N,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,ACT PCPL,DEF,MP,GEN,VIII
4,2,5,8,2,mufoliHuwna,N,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,ACT PCPL,DEF,MP,NOM,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,Ea`biduwna,N,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,ACT PCPL,DEF,MP,NOM,I
3865,110,3,7,1,taw~aAbF[A,N,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,ACT PCPL,INDEF,,ACC,I
3866,111,4,2,1,Ham~aAlapa,N,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,ACT PCPL,DEF,FS,ACC,I
3867,113,3,3,1,gaAsiqK,N,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,ACT PCPL,INDEF,,GEN,I


In [53]:
alladj.drop(['LOCATION', 'FEATURES', 'FEATURES2', 'FEATURESR'], axis=1, inplace=True)

In [54]:
### --- CREATING THE TABLE
cur.execute('''CREATE TABLE IF NOT EXISTS QACN_TABLE
           (unique_id TEXT PRIMARY KEY, sura INTEGER, aya INTEGER, word INTEGER, w_seg INTEGER,  FORM TEXT, TAG TEXT, ROOT TEXT, Lemma TEXT, FORMAr TEXT, ROOTAr TEXT, LemmaAr TEXT, Derivation TEXT, State TEXT, Grop TEXT, Casen TEXT, FormN TEXT )''')


In [55]:
cur.execute('''CREATE TABLE IF NOT EXISTS QACA_TABLE
           (unique_id TEXT PRIMARY KEY, sura INTEGER, aya INTEGER, word INTEGER, w_seg INTEGER,  FORM TEXT, TAG TEXT, ROOT TEXT, Lemma TEXT, FORMAr TEXT, ROOTAr TEXT, LemmaAr TEXT, Derivation TEXT, State TEXT, Grop TEXT, Casen TEXT, FormN TEXT )''')


In [56]:
### --- COPYING THE DATAFRAME TO THE TABLE
alln.to_sql('QACN_TABLE', conn, if_exists='replace', index=False)

3869

In [57]:
### --- COPYING THE DATAFRAME TO THE TABLE
alladj.to_sql('QACA_TABLE', conn, if_exists='replace', index=False)

335

In [58]:
pd.read_sql('select * from QACN_TABLE', conn)

,sura,aya,word,w_seg,FORM,TAG,FORMAr,Root,RootAr,Lemma,LemmaAr,Derivation,State,Grop,Case,FormN
0,1,4,1,1,ma`liki,N,مَٰلِكِ,mlk,ملك,ma`lik,مَٰلِك,ACT PCPL,DEF,M,GEN,I
1,1,7,6,2,magoDuwbi,N,مَغْضُوبِ,gDb,غضب,magoDuwb,مَغْضُوب,PASS PCPL,DEF,M,GEN,I
2,1,7,9,2,D~aA^l~iyna,N,ضَّآلِّينَ,Dll,ضلل,DaA^l~,ضَآلّ,ACT PCPL,DEF,MP,GEN,I
3,2,2,7,3,mut~aqiyna,N,مُتَّقِينَ,wqy,وقي,mut~aqiyn,مُتَّقِين,ACT PCPL,DEF,MP,GEN,VIII
4,2,5,8,2,mufoliHuwna,N,مُفْلِحُونَ,flH,فلح,mufoliHuwn,مُفْلِحُون,ACT PCPL,DEF,MP,NOM,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,109,5,3,1,Ea`biduwna,N,عَٰبِدُونَ,Ebd,عبد,EaAbid,عَابِد,ACT PCPL,DEF,MP,NOM,I
3865,110,3,7,1,taw~aAbF[A,N,تَوَّابًۢا,twb,توب,taw~aAb,تَوَّاب,ACT PCPL,INDEF,,ACC,I
3866,111,4,2,1,Ham~aAlapa,N,حَمَّالَةَ,Hml,حمل,Ham~aAlap,حَمَّالَة,ACT PCPL,DEF,FS,ACC,I
3867,113,3,3,1,gaAsiqK,N,غَاسِقٍ,gsq,غسق,gaAsiq,غَاسِق,ACT PCPL,INDEF,,GEN,I


In [59]:
pd.read_sql('select * from QACA_TABLE', conn)

,sura,aya,word,w_seg,FORM,TAG,FORMAr,Root,RootAr,Lemma,LemmaAr,Derivation,State,Grop,Case,FormN
0,1,6,3,2,musotaqiyma,ADJ,مُسْتَقِيمَ,qwm,قوم,m~usotaqiym,مُّسْتَقِيم,ACT PCPL,DEF,M,ACC,X
1,2,25,31,1,m~uTah~arapN,ADJ,مُّطَهَّرَةٌ,Thr,طهر,m~uTah~arap,مُّطَهَّرَة,PASS PCPL,INDEF,,NOM,II
2,2,65,12,1,xa`si_#iyna,ADJ,خَٰسِـِٔينَ,xsA,خسا,xaAsi},خَاسِئ,ACT PCPL,DEF,MP,ACC,I
3,2,68,15,1,faAriDN,ADJ,فَارِضٌ,frD,فرض,faAriD,فَارِض,ACT PCPL,INDEF,,NOM,I
4,2,69,15,1,faAqiEN,ADJ,فَاقِعٌ,fqE,فقع,faAqiE,فَاقِع,ACT PCPL,INDEF,,NOM,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,101,7,4,1,r~aADiyapK,ADJ,رَّاضِيَةٍ,rDw,رضو,raADiyap,رَاضِيَة,ACT PCPL,INDEF,,GEN,I
331,101,11,2,1,HaAmiyapN[,ADJ,حَامِيَةٌۢ,Hmy,حمي,HaAmiyap,حَامِيَة,ACT PCPL,INDEF,,NOM,I
332,104,6,3,2,muwqadapu,ADJ,مُوقَدَةُ,wqd,وقد,muwqadap,مُوقَدَة,PASS PCPL,DEF,F,NOM,IV
333,104,9,3,1,m~umad~adapK],ADJ,مُّمَدَّدَةٍۭ,mdd,مدد,m~umad~adap,مُّمَدَّدَة,PASS PCPL,INDEF,,GEN,II
